Import necessary libraries

In [2]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from skimage import measure, morphology
from skimage.transform import resize
from scipy.ndimage import binary_closing, binary_opening
from scipy import ndimage
from mayavi import mlab
from skimage import io, filters, exposure
from skimage.restoration import denoise_bilateral
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import IntSlider, Output, VBox
from IPython.display import display

Function to plot 2d slices

In [3]:

def plot_with_ipywidgets(organs_mask, slice_axis=2):
    """
    Visualizza un'immagine volumetrica `organs_mask` con uno slider interattivo
    per cambiare la slice visualizzata.

    Args:
        organs_mask (numpy.ndarray): Volume binario da visualizzare.
        slice_axis (int): Asse lungo il quale effettuare le sezioni (0, 1 o 2).
    """
    # Creare un'area di output separata
    output = Output()

    # Funzione per aggiornare la visualizzazione
    def update(slice_index):
        with output:
            output.clear_output(wait=True)
            plt.figure(figsize=(8, 8))
            if slice_axis == 0:
                slice_data = organs_mask[slice_index, :, :]
            elif slice_axis == 1:
                slice_data = organs_mask[:, slice_index, :]
            else:
                slice_data = organs_mask[:, :, slice_index]
            
            plt.imshow(slice_data, cmap='gray')
            plt.title(f"Slice {slice_index} lungo l'asse {slice_axis}")
            plt.axis('off')
            plt.show()

    # Crea uno slider
    slider = IntSlider(min=0, max=organs_mask.shape[slice_axis] - 1, step=1, value=0)
    slider.observe(lambda change: update(change['new']), names='value')

    # Layout con slider e output
    ui = VBox([slider, output])

    # Inizializza la visualizzazione
    update(0)
    display(ui)


Function to filter lungs

In [4]:
def filtra_polmoni(data_resized):
    organs_mask = (data_resized < -500) & (data_resized > -1000)
    #plot_with_ipywidgets(organs_mask, slice_axis=2)
    organs_mask_closed=np.zeros_like(organs_mask)
    organs_mask_cleaned=np.zeros_like(organs_mask)
    for i in range(organs_mask.shape[2]):
        selem = morphology.disk(3)  # Elemento strutturante sferico di raggio 3
        organs_mask_closed[:,:,i] = binary_closing(organs_mask[:,:,i], structure=selem)
        organs_mask_cleaned[:,:,i] = binary_opening(organs_mask_closed[:,:,i], structure=selem)
    
    labeled_volume, num_features = ndimage.label(organs_mask_cleaned)
    #print(f"Numero di blob trovati: {num_features}")

    blob_properties = measure.regionprops(labeled_volume)

    filtrati = []
    for i, prop in enumerate(blob_properties):
        if prop.area > 100000:
            filtrati.append((prop.area,i + 1))
    filtrati=sorted(filtrati)
    return filtrati[0][1],labeled_volume


def plot_3d_label( labeled_volume, label_number=1,voxel_dimensions=(1, 1, 1)):
    # Crea una maschera binaria per la label specificata
    label_mask = labeled_volume == label_number

    # Trova i contorni della maschera
    verts, faces, _, _ = measure.marching_cubes(label_mask, level=0, spacing=voxel_dimensions)

    # Crea una figura 3D con mlab
    mlab.figure(size=(800, 800))
    mlab.triangular_mesh(verts[:, 0], verts[:, 1], verts[:, 2], faces, color=(1, 0, 0))
    #mlab.title(f"3D Plot della label {label_number}")
    mlab.show()




def dice_score(mask1, mask2):
    """
    Calcola il Dice Score tra due maschere volumetriche.

    Args:
        mask1 (numpy.ndarray): Prima maschera binaria.
        mask2 (numpy.ndarray): Seconda maschera binaria.

    Returns:
        float: Dice Score.
    """
    intersection = np.sum(mask1 * mask2)
    volume_sum = np.sum(mask1) + np.sum(mask2)
    if volume_sum == 0:
        return 1.0  # Se entrambe le maschere sono vuote, il Dice Score è 1
    return 2.0 * intersection / volume_sum


    

   

Read .nii.gz  MRI image

In [ ]:
# Carica il file .nii
img = nib.load('C:\\Users\\alber\\Desktop\\OrganSegmentations\\volume-1.nii.gz')
header = img.header
voxel_dimensions = header.get_zooms()
data = img.get_fdata()

img = nib.load('C:\\Users\\alber\\Desktop\\OrganSegmentations\\labels-1.nii.gz')
true_lung = img.get_fdata()
unique_values = np.unique(true_lung)
#print("Valori presenti in true_lung:", unique_values)
true_lung = np.where(np.isclose(true_lung,3), 1, 0)
#plot_with_ipywidgets(true_lung, slice_axis=2)


Launch filter on image

In [ ]:
label_number, labeled_volume = filtra_polmoni(data)
predicted_lung = np.where(labeled_volume == label_number, 1, 0)
plot_3d_label(predicted_lung,voxel_dimensions=voxel_dimensions)

print(f"Dice Score: {dice_score(true_lung, predicted_lung)}")

# Evaluation

In [ ]:
dice_scores=[]
for i in range(0,131):
    print("Iteration number: ",i)
    img = nib.load('C:\\Users\\alber\\Desktop\\OrganSegmentations\\volume-'+str(i)+'.nii.gz')
    header = img.header
    voxel_dimensions = header.get_zooms()
    data = img.get_fdata()

    img = nib.load('C:\\Users\\alber\\Desktop\\OrganSegmentations\\labels-'+str(i)+'.nii.gz')
    true_lung = img.get_fdata()
    true_lung = np.where(np.isclose(true_lung,3), 1, 0)

    labeled_true_lung, num_features = ndimage.label(true_lung)
    if num_features != 1:
        label_number, labeled_volume = filtra_polmoni(data)
        predicted_lung = np.where(labeled_volume == label_number, 1, 0)
        dice_scores.append(dice_score(true_lung, predicted_lung))
    else:
        print(f"Volume {i} has separated lungs")
    

print(f"Average Dice Score: {np.mean(dice_scores)}")
